# Building a Chromosome for `shadie`

The simplest way to prepare a SLiM simulation using `shadie` is to let the `Chromosome` class automatically generate a single gene. The gene will consist of a single exon by default, although the user can also specify number of exons. The user can also supply a length (in base pairs) for the `genome_size` argument. 

In [13]:
import shadie

In [ ]:
shadie.

In [16]:
shadie.chrom.random()
shadie.chrom.explicit()
shadie.chrom.standard()

AttributeError: module 'shadie' has no attribute 'chrom'

In [ ]:
shadie.build.random()

In [12]:
shadie.build.ChromosomeRandoma


shadie.build.ChromosomeBase

In [11]:
c = shadie..standard()
c.data
c.inspect()

AttributeError: module 'shadie.chromosome' has no attribute 'standard'

In [3]:
from shadie.build import *

In [6]:
ChromosomeStandard().data.head()

,name,start,end,eltype,script
0,noncds,0,2000,g3,"<ElementType: noncds, g3, ['m1'], [1], mmJukes..."
2000,exon,2000,4000,g1,"<ElementType: exon, g1, ['m3', 'm4'], (8, 0.1)..."
4000,intron,4000,6000,g2,"<ElementType: intron, g2, ['m3'], [1], mmJukes..."
6000,exon,6000,8000,g1,"<ElementType: exon, g1, ['m3', 'm4'], (8, 0.1)..."
8000,noncds,8000,10000,g3,"<ElementType: noncds, g3, ['m1'], [1], mmJukes..."


In [4]:
ChromosomeExplicit({50: INTRON, 100: EXON, 150: INTRON}).data

,name,start,end,eltype,script
0,intron,0,50,g2,"<ElementType: intron, g2, ['m3'], [1], mmJukes..."
50,exon,50,150,g1,"<ElementType: exon, g1, ['m3', 'm4'], (8, 0.1)..."
150,intron,150,300,g2,"<ElementType: intron, g2, ['m3'], [1], mmJukes..."


In [151]:
class ChromosomeBase:
    def __init__(self, genome_size):
        self.genome_size = genome_size
        self.data = pd.DataFrame(
            columns=['type', 'name', 'start', 'finish', 'eltype', 'script'],
            data=None,
        )       


class Random(ChromosomeBase):
    def __init__(self, genome_size=20000, seed=None):
        super().__init__(genome_size)
        self.rng = np.random.default_rng(seed)
        # self.intron = INTRON
        # self.exon = EXON
        # self.noncds = NONCDS


    def get_noncds_span(self, scale=5000):
        """
        Draws the number of bases until the next element from an 
        exponential distribution. The scale is the average waiting
        time in number of bp.
        """
        return int(self.rng.exponential(scale=scale))


    def get_cds_spans(self, length_scale=1000, intron_scale=1000):
        """
        Draws the number of exons in a fixed length space from a 
        Poisson distribution. The lam parameter is the average number
        of events per sampled region. A value of 0.005 means one intron
        per 200bp.
        """
        cds_span = int(self.rng.exponential(scale=length_scale))
        n_introns = int(self.rng.poisson(cds_span / intron_scale))
        if n_introns:
            splits = self.rng.dirichlet(np.ones(n_introns * 2 - 1))
            splits = (splits * cds_span).astype(int)
            splits[-1] = cds_span - sum(splits[:-1])
        else:
            splits = np.array([cds_span])
        return splits

    def run(self):
        """
        ...
        """
        idx = 0
        
        while 1:
            # get non-cds span
            pos = self.get_noncds_span()
            self.data.loc[idx] = (
                "noncoding", 
                None, 
                idx, 
                min(idx + pos, self.genome_size), 
                self.noncds.name, 
                self.noncds,
            )
            idx += pos
            
            # get cds span
            posses = self.get_cds_spans()

            # break if cds goes beyond the end of the genome.
            if idx + posses.sum() > self.genome_size:
                break

            # enter the cds into data
            for enum, pos in enumerate(posses):
                if not enum % 2:
                    self.data.loc[idx] = (
                        "exon", 
                        None, 
                        idx, 
                        idx + pos, 
                        self.exon.name, 
                        self.exon,
                    )
                else:
                    self.data.loc[idx] = (
                        "intron", 
                        None, 
                        idx, 
                        idx + pos, 
                        self.intron.name, 
                        self.intron,
                    )
                idx += pos


In [152]:
m0 = shadie.mtype(0.5, 'n', 1.0, 0.5)
m1 = shadie.mtype(0.5, 'n', 2.0, 1.0)
m2 = shadie.mtype(0.5, 'f', 0.0)
e0 = shadie.etype([m0, m1], [0.5, 1.0])
e1 = shadie.etype([m2], [1.0])
l0 = shadie.elist(e0, e1)

In [154]:
c = Random(100000)

In [155]:
c.intron = e1
c.exon = e0
c.noncds = e1

In [162]:
c.run()

In [165]:
c.data.script[0]

<ElementType: None, g48, ['m72'], [1.0], mmJukesCantor(1e-09/3)>

In [90]:
c.get_cds_spans().sum()

90

In [79]:
idx = 0
while 1:
    
    # get noncds span
    pos = c.get_span(5000)
    
    # enter noncds span
    c.data.loc[idx] = "noncoding", None, idx, pos, c.noncds.name, c.noncds

    # get cds span
    pos = c.get_span(1000)

    # split cds into introns
    nintrons = c.get_n_introns_in_cds(1 / 300.)
    splits = c.get_cds_spans(pos, nintrons)

In [362]:
np.random.exponential(1000)

402.60767324553086

In [386]:
np.random.poisson(1000 * 1/1000)

1

In [405]:
xs = np.random.dirichlet(np.ones(3))

In [409]:
(xs * 1000).astype(int)

array([ 80, 428, 491])

In [411]:
80 + 428 + 491

999

In [106]:
idx = 0
while 1:
    
pos = 500
c.data.loc[0] = "noncoding", None, idx, pos, c.noncds.name, c.noncds
idx = 500
pos = 800
c.data.loc[500] = "exon", None, idx, pos, c.exon.name, c.exon
c.data

,type,name,start,finish,eltype,script
0,noncoding,None,0,500,g4,"<ElementType: None, g4, ['m6'], [1.0], mmJukes..."
500,exon,None,500,800,g3,"<ElementType: None, g3, ['m4', 'm5'], [0.5, 1...."


### Mutations

In [3]:
NEUTRAL = shadie.mtype(0.5, 'f', 0.0)
SYNONYMOUS = shadie.mtype(0.5, 'f', 0.00)
DELETERIOUS = shadie.mtype(0.5, 'g', -2.0, 0.05)
BENEFICIAL = shadie.mtype(0.5, 'e', 0.5)

mlist = shadie.mlist(DELETERIOUS, BENEFICIAL).draw(width=400);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="225.0px" viewBox="0 0 400.0 225.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9d4c6f4877e5489088a19ddd0713aa41"> <polygon points="50.0,175.0 50.36750058245772,175.0 50.73500116491546,175.0 51.10250174737318,175.0 51.470002329830905,175.0 51.837502912288635,175.0 52.205003494746364,175.0 52.57250407720409,175.0 52.94000465966181,175.0 53.30750524211955,175.0 53.67500582457727,175.0 54.04250640703499,175.0 54.41000698949272,175.0 54.77750757195045,175.0 55.145008154408174,175.0 55.512508736865904,175.0 55.880009319323634,175.0 56.247509901781356,175.0 56.61501048423908,175.0 56.982511066696816,175.0 57.35001164915454,175.0 57.71751223161227,175.0 58.08501281406999,175.0 58.45251339652772,175.0 58.82001397898544,175.0 59.187514561443166,175.0 59.5550151439009,175.0 59.922515726358625,175.0 60.29001630881635,175.0 60.657516891274085,175.0 61.02501747373181,175.0 61.39251805618953,175.0 61.76001863864726,175.0 62.12751922110499,175.0 62.49501980356272,175.0 62.86252038602044,175.0 63.23002096847817,175.0 63.5975215509359,175.0 63.965022133393624,175.0 64.33252271585135,175.0 64.70002329830908,175.0 65.0675238807668,175.0 65.43502446322452,175.0 65.80252504568226,175.0 66.17002562813998,175.0 66.53752621059772,175.0 66.90502679305544,175.0 67.27252737551316,175.0 67.64002795797089,175.0 68.00752854042862,175.0 68.37502912288635,175.0 68.74252970534407,175.0 69.1100302878018,175.0 69.47753087025953,175.0 69.84503145271725,175.0 70.21253203517497,175.0 70.58003261763271,175.0 70.94753320009045,175.0 71.31503378254816,175.0 71.68253436500588,175.0 72.05003494746362,175.0 72.41753552992134,175.0 72.78503611237906,175.0 73.1525366948368,175.0 73.52003727729453,175.0 73.88753785975226,175.0 74.25503844220998,175.0 74.6225390246677,175.0 74.99003960712544,175.0 75.35754018958316,175.0 75.7250407720409,175.0 76.09254135449862,175.0 76.46004193695634,175.0 76.82754251941407,175.0 77.19504310187179,175.0 77.56254368432951,175.0 77.93004426678723,175.0 78.29754484924499,175.0 78.66504543170271,175.0 79.03254601416043,175.0 79.40004659661815,175.0 79.76754717907588,175.0 80.13504776153361,175.0 80.50254834399134,175.0 80.87004892644907,175.0 81.2375495089068,175.0 81.60505009136452,175.0 81.97255067382224,175.0 82.34005125627996,175.0 82.7075518387377,175.0 83.07505242119544,175.0 83.44255300365316,175.0 83.81005358611088,175.0 84.1775541685686,175.0 84.54505475102633,175.0 84.91255533348405,175.0 85.28005591594177,175.0 85.64755649839952,175.0 86.01505708085725,175.0 86.38255766331497,175.0 86.38255766331497,143.29042914485206 86.01505708085725,124.49515672143893 85.64755649839952,108.54273467784756 85.28005591594177,95.12321893517102 84.91255533348405,83.95641861596195 84.54505475102633,74.78922837318171 84.1775541685686,67.39318991038678 83.81005358611088,61.562263569426534 83.44255300365316,57.11079242402139 83.07505242119544,53.87164275414 82.7075518387377,51.69450609705358 82.34005125627996,50.44434928550015 81.97255067382224,50.0 81.60505009136452,50.25285638881904 81.2375495089068,51.105710252556705 80.87004892644907,52.47167415746021 80.50254834399134,54.273203638421904 80.13504776153361,56.44120638482232 79.76754717907588,58.91423097509921 79.40004659661815,61.637728343920195 79.03254601416043,64.56337973349194 78.66504543170271,67.64848540189371 78.29754484924499,70.85540884009137 77.93004426678723,74.15107168889543 77.56254368432951,77.50649495072088 77.19504310187179,80.89638246149238 76.82754251941407,84.29874292808081 76.46004193695634,87.69454714871955 76.09254135449862,91.06741732018767 75.725040772040

### Elements

In [4]:
INTRON = shadie.etype([NEUTRAL], [1])
EXON = shadie.etype([SYNONYMOUS, DELETERIOUS, BENEFICIAL], [33, 66, 1])
NONCDS = shadie.etype([NEUTRAL], [1], mutation_rate=0.0)

In [106]:
import numpy as np

In [220]:
rng = np.random.default_rng()

In [228]:
rng.poisson(lam=1/100)

0

In [117]:
rng.exponential(scale=100.0)

94.49656093528965

In [104]:
scipy.stats.expon.rvs(loc=0, scale=100, size=10000).mean()

101.49552365619441

In [105]:
scipy.stats.poisson.rvs(loc=0, mu=100, size=10000).mean()

99.9187

### Chromosome

In [6]:
# chromosome.random()
# chromosome.explicit()

In [7]:
class Build:
    def __init__(self, introns, exons, noncds):
        self.introns = introns
        self.exons = exons
        self.noncds = noncds

In [37]:
bld = Build(INTRON, EXON, NONCDS)
bld

In [2]:
Chromosome(100).random()

In [2]:
import io

In [ ]:
one_gene = Chromosome(2000)

In [2]:
#create Chromosome object. If no "genome" argument is supplied, the Chromosome class will generate a single gene
one_gene = Chromosome(genome_size = 2000)

01:06 | INFO    | genes           | Chromosome complete!


In [3]:
#the Chromosome object can now be inspected using review() function
Chromosome.review(one_gene, item = "mutations")
Chromosome.review(one_gene, item = "eltypes")
Chromosome.review(one_gene, item = "elements")

#simpler syntax
one_gene.review("chromosome")

Mutation Types:
 <MutationList: ['m1', 'm2', 'm3', 'm4']> 

Genomic Element Types:
 <ElementList: ['g1', 'g3']> 

Genomic Elements:



,type,name,start,finish,eltype,script
0,noncoding,NaN,0,308,g3,"<ElementType: 'None', g3, ['m1'], [1], mmJukes..."
309,exon,NaN,309,1409,g1,"<ElementType: 'None', g1, ['m3', 'm4'], [8, 0...."
1410,noncoding,NaN,1410,1999,g3,"<ElementType: 'None', g3, ['m1'], [1], mmJukes..."


Chromosome Summary
# of Genes: 1
Average # exons per gene: 1.0
Average exon length: 1101.0 nt
Average # introns per gene: 0.0
Average introns length: 0 nt

Static Chromosome Plot:



<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="3000.0px" height="200.0px" viewBox="0 0 3000.0 200.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1ee55de280c54754995649aa3f85da16"> 0 500 1000 1500 2000 0.0 0.5 1.0

## Random Chromosome
You can also use the `Build` class of `shadie` to generate a random chromosome for you:

In [4]:
from shadie import Build

In [5]:
#create a Build class object
random_chromosome = Build()

#run the random() function to generate the chromosome
Build.random(random_chromosome)

01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO 

01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO    | random          | Gene added
01:07 | INFO 

In [6]:
print(random_chromosome.mutationlist)

<MutationList: ['m1', 'm2', 'm3', 'm4']>


### Pass Build object to Chromosome Class
`random_chromsome` is a Build class object that can be assigned to `genome` argument in `Chromosome` class:

In [7]:
final_chromosome = Chromosome(genome = random_chromosome)
final_chromosome.review("elements")

Genomic Elements:



,type,name,start,finish,eltype,script
0,noncoding,NaN,0,1794,g3,"<ElementType: 'None', g3, ['m1'], [1], mmJukes..."
1795,exon,NaN,1795,2103,g1,"<ElementType: 'None', g1, ['m3', 'm4'], [8, 0...."
2104,intron,NaN,2104,2644,g2,"<ElementType: 'None', g2, ['m3'], [1], mmJukes..."
2645,exon,NaN,2645,2815,g1,"<ElementType: 'None', g1, ['m3', 'm4'], [8, 0...."
2816,intron,NaN,2816,3289,g2,"<ElementType: 'None', g2, ['m3'], [1], mmJukes..."
...,...,...,...,...,...,...
1001810,intron,NaN,1001810,1002336,g2,"<ElementType: 'None', g2, ['m3'], [1], mmJukes..."
1002337,exon,NaN,1002337,1002637,g1,"<ElementType: 'None', g1, ['m3', 'm4'], [8, 0...."
1002638,intron,NaN,1002638,1003007,g2,"<ElementType: 'None', g2, ['m3'], [1], mmJukes..."
1003008,exon,NaN,1003008,1003236,g1,"<ElementType: 'None', g1, ['m3', 'm4'], [8, 0...."


In [8]:
final_chromosome.review("chromosome")

Chromosome Summary
# of Genes: 205
Average # exons per gene: 3.8048780487804876
Average exon length: 258.37179487179486 nt
Average # introns per gene: 2.8048780487804876
Average introns length: 454.95652173913044 nt

Static Chromosome Plot:



<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="3000.0px" height="200.0px" viewBox="0 0 3000.0 200.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2330a57963934d8fa84c9000265c457a"> <rect class="toyplot-Datum" x="1236.1175375310122" y="50.0" width="1.3065706308677818" height="100.0" style="fill:rgb(27.5%,5

### Interactive Plot
You can use the `review()` function to generate an interactive plot to inspect your chromosome structure further. Draw a region on the bottom plot to view that region in the top plot. 

In [9]:
final_chromosome.review("interactive")

Interactive altair chromosome map:


## Random Chromosome with Custom Mutation Types & Genomic Element Types 

In [16]:
from shadie import ElementList
from shadie import ElementType
from shadie import MutationType
from shadie import MutationList

#create your custom mutation types and save to a MutationList
mut1 = MutationType(0.5, "f", 0)
mut2 = MutationType(0.5, "e", 0.4)
mut3 = MutationType(0.5, "n", 0.4, .1)
mut4 = MutationType(0.5, "w", 0.3, 0.2)
mut5 = MutationType(0.5, "g", -0.4, .1)

mutlist = MutationList(mut1, mut2, mut3, mut4, mut5)

#create your custom genomic element types and save to an ElementList
noncod = ElementType(mut1, 1, altname = "nc")
exon1 = ElementType([mut2, mut5], [1, 1], altname = "ex1")
exon2 = ElementType([mut2, mut3, mut4], [9, 1, .02], altname = "ex2")
intron1 = ElementType([mut2, mut5], [1, 1], altname = "int1")
intron2 = ElementType([mut2, mut5], [1, 1], altname = "int2")

mycustomlist = ElementList(mutlist, noncod, exon1, exon2, intron1, intron2)

01:50 | WARNING | __init__        | 'g', 'n', and 'w' distributions take 2 params
01:50 | WARNING | __init__        | 'g', 'n', and 'w' distributions take 2 params
01:50 | WARNING | __init__        | 'e' and 'f' distributions take 1 param
01:50 | WARNING | __init__        | 'e' and 'f' distributions take 1 param
01:50 | WARNING | __init__        | 'e' and 'f' distributions take 1 param


In [17]:
from shadie import Build

#initialize the Build class object
custom_build = Build(
    exons = [exon1, exon2], 
    introns = [intron1, intron2], 
    noncoding = [noncod], 
    elementlist = mycustomlist)

In [18]:
#run the same random function
custom_build.random()

01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO 

01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO 

01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Gene added
01:50 | INFO    | random          | Chromosome complete!


In [19]:
from shadie import Chromosome

customized_chrom = Chromosome(genome = custom_build)

In [20]:
customized_chrom.review("elements")

Genomic Elements:



,name,start,finish,eltype,script,type
0,nc,0,3847,g5,"'g5', c(m5),c(1), mmJukesCantor(1e-06/3)",noncoding
3848,ex1,3848,4063,g6,"'g6', c(m6, m9),c(1, 1), mmJukesCantor(1e-06/3)",exon
4064,int1,4064,4639,g8,"'g8', c(m6, m9),c(1, 1), mmJukesCantor(1e-06/3)",intron
4640,ex2,4640,4900,g7,"'g7', c(m6, m7, m8),c(9, 1, 0.02), mmJukesCant...",exon
4901,nc,4901,5780,g5,"'g5', c(m5),c(1), mmJukesCantor(1e-06/3)",noncoding
...,...,...,...,...,...,...
999020,int2,999020,999305,g9,"'g9', c(m6, m9),c(1, 1), mmJukesCantor(1e-06/3)",intron
999306,ex1,999306,999556,g6,"'g6', c(m6, m9),c(1, 1), mmJukesCantor(1e-06/3)",exon
999557,int1,999557,1000162,g8,"'g8', c(m6, m9),c(1, 1), mmJukesCantor(1e-06/3)",intron
1000163,ex1,1000163,1000346,g6,"'g6', c(m6, m9),c(1, 1), mmJukesCantor(1e-06/3)",exon


In [21]:
import pandas as pd
final_chromosome.genome["name"]

for index, row in final_chromosome.genome.iterrows():
    if pd.isna(row['name']):
        print(row['type'])
    else: 
        print(row['name'])

noncoding
exon
intron
exon
noncoding
exon
noncoding
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
noncoding
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
nonc

exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
noncoding
exon
noncoding
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
noncoding
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
noncoding
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
intron
exon
noncoding
exon
intron
exon
intron
exon
noncodin

In [22]:
customized_chrom.review("chromosome")

Chromosome Summary
# of Genes: 220
Average # exons per gene: 3.8727272727272726
Average exon length: 259.46596244131456 nt
Average # introns per gene: 2.8727272727272726
Average introns length: 452.40822784810126 nt

Static Chromosome Plot:



<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="3000.0px" height="200.0px" viewBox="0 0 3000.0 200.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t41e8998361944cd188a69d0495f63082"> <rect class="toyplot-Datum" x="1190.4874508545533" y="50.0" width="1.011

In [23]:
customized_chrom.review("interactive")

Interactive altair chromosome map:


## Custom Chromosome
Finally, you can define your own chromosome by providing a pandas dataframe that contains, at minimum, the `name` of the genomic element (as defined by you *or* using `shadie` defaults), the `start base` and the `end base`. Alternatively, you can provide the internal `idx` of the genomic element (in place of the `name`). You must provide a GenomeList class object and your genomic elements must be defined in the list. 